In [3]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import sys
import gym
import numpy as np
import random
import math
from collections import defaultdict, deque
import matplotlib.pyplot as plt
%matplotlib tk

#
from agent import Agent
from monitor import interact
import gym
import numpy as np

from tqdm import tqdm


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [4]:

# ### Actions
# There are 6 discrete deterministic actions:
# - 0: move south
# - 1: move north
# - 2: move east
# - 3: move west
# - 4: pickup passenger
# - 5: drop off passenger

# Passenger locations/Destinations:
# - 0: R(ed)
# - 1: G(reen)
# - 2: Y(ellow)
# - 3: B(lue)
# - 4: in taxi

#
# hierarchy of states;
# num_rows = 5
# num_columns = 5
# locs = env.locs
# num_actions = 6
# for row in range(num_rows):
#     for col in range(num_columns):
#         for pass_idx in range(len(locs) + 1):  # +1 for being inside taxi
#             for dest_idx in range(len(locs)):
#                 state = env.encode(row, col, pass_idx, dest_idx)
#                 print ("pass idx: ", pass_idx, ", dest: ", dest_idx, ", state: ", state)
#                 for action in range(num_actions):
#                     pass
                
#                     #self.P[state][action].append(
#                     #            (1.0, new_state, reward, terminated)
#                     #        )
#


In [16]:
######################################################
################## TRAIN AGENT #######################
######################################################

#
env = gym.make('Taxi-v3')
nA = 6
epsilon = 0.25
epsilon_scaling = 1.000015
gamma = 1.0
alpha = 0.2
agent = Agent(nA,
              epsilon,
              epsilon_scaling,
              alpha,
              gamma)

#
###########################################
###########################################
###########################################
verbose = False

# trianing state
agent.greedy_policy=False

#
current_reward = 0
last_reward = 0
i_episodes = np.arange(1000000)

#
with tqdm(i_episodes) as pbar:
    
    # loop over episodes
    for i_episode in pbar:
        
        #
        env.reset()
        
        #
        #env.s = 0

        #
        agent.i_episode = i_episode

        # figrue out where agent and target is inthe world
        taxi_row, taxi_col, pass_idx, dest_idx = env.decode(env.s)

   
        # loop until we pickup the passanger
        state_t0 = env.s
   
        #
        action_t0 = agent.select_action(state_t0)

        #
        ctr = 0
        #pickup_flag = True
        while True:

            #
            #print ('sampled action: ', action_t0)

            # take action in enviorment
            state_t1, reward_t1, done, action_mask_t1 = env.step(action_t0)
            taxi_row, taxi_col, pass_idx, dest_idx = env.decode(env.s)

            #
            agent.action_mask_t1 = action_mask_t1['action_mask']


            # update Q of agent
            action_t1 = agent.step_sarsa(state_t0,
                                   action_t0,
                                   reward_t1,
                                   state_t1,
                                   #action_t1,
                                   done)

            #
            state_t0 = state_t1
            action_t0 = action_t1

            #
            if done==True:
                if verbose:
                    if ctr<199:
                        print ("copmleted run ", i_episode, " ... in # steps: ", ctr)
                break

            ctr+=1
            current_reward+=reward_t1

        #
        pbar.set_postfix(average_reward=current_reward, refresh=False)
        pbar.update(0)
        current_reward = 0

print ('done')

100%|██████████████████████████████████████████| 1000000/1000000 [2:50:45<00:00, 97.60it/s, average_reward=-142]

done


In [ ]:
######################################################
################### TEST AGENT #######################
######################################################
#
n_episodes = 1000
window = 100

agent.greedy_policy=True
agent.step = agent.step_sarsa

# force learning of a single state:
env.s = 0
env.fixed_state=0

#
avg_rewards, best_avg_reward = interact(env, 
                                        agent,
                                        n_episodes,
                                        window
                                       )

#print ("avg reward: ", avg_rewards)
print ("best_avg_reward: ", best_avg_reward)

Episode 244/1000 || Best average reward -354.53

KeyboardInterrupt: 